In [ ]:
import os
import random
import numpy as np
import skimage
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras import backend as keras
from keras.models import load_model
from keras.utils import to_categorical
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed(12345)
print(np.__version__)

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive


import os
os.chdir("/content/drive/Colab Notebooks")

import numpy as np
import scipy.io as io
mat=io.loadmat('TestTest8.mat', squeeze_me=True)
np.save('TestTest8.npy',mat)

import numpy as np
import scipy.io as io
matt=io.loadmat('TestTest9.mat', squeeze_me=True)
np.save('TestTest9.npy',matt)

b = np.zeros(shape=(2000,32,32))

bb = np.zeros(shape=(2000,32,32,2))

for i in range(0, 2000):
    #print(i)
    pp=mat['koo'][i]
    b[i,:,:]=pp

for i in range(0, 2000):
    #print(i)
    pp=matt['TestTest9'][i][0]
    bb[i,:,:,0]=pp
    pp=matt['TestTest9'][i][1]
    bb[i,:,:,1]=pp

dd = np.zeros(shape=(2000,64,64,2))

import cv2
for i in range(0, 2000):
    #print(i)
   dd[i,:,:,0] = cv2.resize(bb[i,:,:,0],(64,64))
   dd[i,:,:,1] = cv2.resize(bb[i,:,:,1],(64,64))

d = np.zeros(shape=(2000,64,64))

import cv2
for i in range(0, 2000):
    #print(i)
   d[i,:,:] = cv2.resize(b[i,:,:],(64,64))

data = d
label = dd

j = 1793
#plt.figure(figsize=(4,2),dpi=200)
plt.subplot(1,2,1)
plt.imshow(np.squeeze(data[j,:,:]),cmap=plt.cm.gray)
plt.subplot(1,2,2)
plt.imshow(np.squeeze(label[j,:,:,0]),interpolation="bilinear",vmin=0.2,vmax=1.0,cmap=plt.cm.gray)
#figname = './fig/data_' +str(j)+'.jpg'
#plt.savefig(figname)

def Z_ScoreNormalization(x):
    for i in range(len(x)):
        mu = np.average(x[i,:])
        sigma = np.std(x[i,:])
        x[i,:] = (x[i,:] - mu) / sigma;
    return x;

# normalize data
data = Z_ScoreNormalization(data)
data = np.reshape(data,(2000,64,64,1))
label = np.reshape(label,(2000,64,64,2))
# split the train_data and val_data
train_data = data[0:1600]
val_data = data[1600:2000]
train_label = label[0:1600]
val_label = label[1600:2000]
print(train_data.shape)
print(val_data.shape)
print(train_label.shape)
print(val_label.shape)

from __future__ import print_function

from keras.models import Model
from keras.layers import Input, MaxPooling2D, UpSampling2D, Dropout, Conv2D, Concatenate, Activation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2


# define conv_factory: batch normalization + ReLU + Conv2D + Dropout (optional)
def conv_factory(x, concat_axis, nb_filter,
                 dropout_rate=None, weight_decay=1E-4):
    x = BatchNormalization(axis=concat_axis,
                           gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (5, 5), dilation_rate=(2, 2),
               kernel_initializer="he_uniform",
               padding="same",
               kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


# define dense block
def denseblock(x, concat_axis, nb_layers, growth_rate,
               dropout_rate=None, weight_decay=1E-4):
    list_feat = [x]
    for i in range(nb_layers):
        x = conv_factory(x, concat_axis, growth_rate,
                         dropout_rate, weight_decay)
        list_feat.append(x)
        x = Concatenate(axis=concat_axis)(list_feat)

    return x


# define model U-net modified with dense block
def get_model_deep_speckle():
    inputs = Input((64, 64, 1))
    print("inputs shape:", inputs.shape)

    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    print("conv1 shape:", conv1.shape)
    db1 = denseblock(x=conv1, concat_axis=3, nb_layers=4, growth_rate=16, dropout_rate=0.5)
    print("db1 shape:", db1.shape)
    pool1 = MaxPooling2D(pool_size=(2, 2))(db1)
    print("pool1 shape:", pool1.shape)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    print("conv2 shape:", conv2.shape)
    db2 = denseblock(x=conv2, concat_axis=3, nb_layers=4, growth_rate=16, dropout_rate=0.5)
    print("db2 shape:", db2.shape)
    pool2 = MaxPooling2D(pool_size=(2, 2))(db2)
    print("pool2 shape:", pool2.shape)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    print("conv3 shape:", conv3.shape)
    db3 = denseblock(x=conv3, concat_axis=3, nb_layers=4, growth_rate=16, dropout_rate=0.5)
    print("db3 shape:", db3.shape)
    pool3 = MaxPooling2D(pool_size=(2, 2))(db3)
    print("pool3 shape:", pool3.shape)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    print("conv4 shape:", conv4.shape)
    db4 = denseblock(x=conv4, concat_axis=3, nb_layers=4, growth_rate=16, dropout_rate=0.5)
    print("db4 shape:", db4.shape)
    pool4 = MaxPooling2D(pool_size=(2, 2))(db4)
    print("pool4 shape:", pool4.shape)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    print("conv5 shape:", conv5.shape)
    db5 = denseblock(x=conv5, concat_axis=3, nb_layers=4, growth_rate=16, dropout_rate=0.5)
    print("db5 shape:", db5.shape)
    up5 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(db5))
    print("up5 shape:", up5.shape)
    merge5 = Concatenate(axis=3)([db4, up5])
    print("merge5 shape:", merge5.shape)

    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    print("conv6 shape:", conv6.shape)
    db6 = denseblock(x=conv6, concat_axis=3, nb_layers=3, growth_rate=16, dropout_rate=0.5)
    print("db5 shape:", db6.shape)
    up6 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(db6))
    print("up6 shape:", up6.shape)
    merge6 = Concatenate(axis=3)([db3, up6])
    print("merge6 shape:", merge6.shape)

    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    print("conv7 shape:", conv7.shape)
    db7 = denseblock(x=conv7, concat_axis=3, nb_layers=3, growth_rate=16, dropout_rate=0.5)
    print("db7 shape:", db7.shape)
    up7 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(db7))
    print("up7 shape:", up7.shape)
    merge7 = Concatenate(axis=3)([db2, up7])
    print("merge7 shape:", merge7.shape)

    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    print("conv8 shape:", conv8.shape)
    db8 = denseblock(x=conv8, concat_axis=3, nb_layers=3, growth_rate=16, dropout_rate=0.5)
    print("db8 shape:", db8.shape)
    up8 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(db8))
    print("up8 shape:", up8.shape)
    merge8 = Concatenate(axis=3)([db1, up8])
    print("merge8 shape:", merge8.shape)

    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    print("conv9 shape:", conv9.shape)
    db9 = denseblock(x=conv9, concat_axis=3, nb_layers=3, growth_rate=16, dropout_rate=0.5)
    print("db9 shape:", db9.shape)
    conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(db9)
    print("conv10 shape:", conv10.shape)
    conv11 = Conv2D(2, 1, activation='softmax')(conv10)
    print("conv11 shape:", conv11.shape)

    model = Model(inputs=inputs, outputs=conv11)

    return model

model = get_model_deep_speckle()

#model = unet()
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy',metrics=['accuracy'])
# checkpoint
filepath="check1channel/fault.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy',verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]
print("data prepared, ready to train!")
# Fit the model
history = model.fit(train_data, train_label,
#                     validation_split=0.2,
                    validation_data=(val_data,val_label),
                    epochs=10000,
                    batch_size=20,
                    shuffle=True,
                    verbose=1,callbacks=callbacks_list)